In [1]:
import pandas as pd
from bokeh.io import show, output_notebook
from bokeh.palettes import OrRd
from bokeh.plotting import figure
from bokeh.transform import transform
from bokeh.models import ColumnDataSource, LinearColorMapper
from datetime import datetime
import numpy as np

output_notebook()

Loading BokehJS ...

In [9]:
def plotcal(
    df, x, y, day_of_month_column,
    color_column,
    hover_tooltips=None,
    text_color='grey',
    text_font='courier',
    palette=OrRd[9],
    weekdays=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'],
    mode='calendar',
    fig_args={
        'plot_width':300,
        'plot_height':200,
        'toolbar_location':None,
        'tools':'hover',
        'x_axis_location':"above"
    },
    rect_args={
        'width':1,
        'height':1,
        'line_color':'white',
        'line_width':0,
    },

    show_dates=True

    ):
    
    source = ColumnDataSource(df)
    mapper = LinearColorMapper(palette=palette, low=df[color_column].max(), high=df[color_column].min())

    if mode == 'calendar':
        range_args ={
            'x_range':weekdays,
            'y_range':list(reversed(list(df[y].unique())))
        }
        xy = {'x':x, 'y':y}

    elif mode == 'github':
        range_args ={
            'x_range':list(df[y].unique()),
            'y_range':weekdays
        }
        xy = {'x':y, 'y':x}
    
    p = figure(**{**fig_args, **range_args})

    rect_renderer = p.rect(
        **{**rect_args, **xy}, 
        source=source, 
        fill_color=transform(color_column, mapper)
    )

    if show_dates:
        text_renderer = p.text(
            **xy, 
            text=day_of_month_column,
            text_align='center',
            text_baseline='middle',
            text_color=text_color,
            text_font=text_font,
            source=source
        )

    if range_args['x_range'] != weekdays:
        p.xaxis.major_label_text_font_size = '0px'

    if range_args['y_range'] != weekdays:
        p.yaxis.major_label_text_font_size = '0pt'

    p.axis.major_label_text_color=text_color
    p.axis.axis_line_color = None
    p.axis.major_tick_line_color = None
    p.outline_line_color = None
    p.grid.grid_line_color = None
    p.axis.axis_line_color = None
    p.axis.major_tick_line_color = None
    p.axis.major_label_standoff = 0
    p.hover.tooltips = hover_tooltips
    rect_renderer.nonselection_glyph.fill_alpha=1

    return p, source

# Generate data

In [10]:
df = pd.DataFrame(
    {'date': pd.date_range('2020-03-01','2020-12-31')}
)

df['Day'] = df['date'].dt.strftime('%a')
df['dom'] = df['date'].dt.day
df['Week'] = df['date'].dt.isocalendar().week
df['Week'] = (df['Week'] - df['Week'].min()).astype(str)
df['val'] = np.random.rand(len(df))
df['date_str'] = df['date'].dt.strftime('%a %b %d %y')

df = df.set_index('date')

# Calendar plots

In [11]:
p, _ = plotcal(
    df, 
    x='Day', y='Week', day_of_month_column='dom', 
    color_column='val',
    mode='github', 
    fig_args={
        'plot_width':900,
        'plot_height':175,
        'tools':'hover',
        'toolbar_location':None,
    }, 
    hover_tooltips=[
        ('Value','@val'),
        ('Date','@date_str')
    ], 
    show_dates=False
)
show(p)

In [13]:
p, _ = plotcal(
    df.loc[datetime.today() - pd.Timedelta('31 d'): datetime.today()], 
    x='Day', y='Week', day_of_month_column='dom', 
    color_column='val',
    mode='calendar',   
    fig_args={
        'plot_width':400,
        'plot_height':200,
        'tools':'hover',
        'toolbar_location':None,
        'x_axis_location':"above"
    }, 
    hover_tooltips=[
        ('Value','@val'),
        ('Date','@date_str')
    ], 
    show_dates=True
)
show(p)